## Environment Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

from tqdm import tqdm_notebook as tqdm

from preprocessing import *

from scipy.sparse import find


In [ ]:
fname = '../data/ntup/partGun_PDGid15_x1000_Pt3.0To100.0_NTUP_1.root'
rootfile = uproot.open(fname)['ana']['hgc']
figs = []

preprocessing_algo = make_graph_etaphi
grouping_algo = 'knn' #or 'kdtree'
preprocessing_args= dict(k=4)
#preprocessing_args= dict(r = 0.07) #if algo == 'kdtree'
layer_norm = 150 #only used for etaphi, no effect for other preprocessors

In [ ]:
def plotHist(axes, data, xlabel, ylabel, title, Nbins = 100, range=None, xlog=False, ylog=False):
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_title(title)
    if xlog:
        axes.set_xscale('log')
        Nbins = np.logspace(np.log10(data.min()),np.log10(data.max()),Nbins)
    return axes.hist(data, bins=Nbins, range=range, histtype='step', log=ylog); 
    
def plotHist_absxlog(axes, data, xlabel, ylabel, title, Nbins = 100, ylog=False):
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_title(title)
    axes.set_xscale('log')
    Nbins = np.logspace(np.log10(np.abs(data).min()),np.log10(np.abs(data).max()),Nbins)
    axes.hist(data, bins=Nbins, histtype='step', log=ylog); 
    
def plotHist_layers(data, ylabel, title, xlabel="Layer", log=True):
    fig,axes = plt.subplots(figsize=(10, 7));
    axes.set_xlabel(xlabel)
    axes.set_xticks(np.arange(53)+0.5, minor=True)
    axes.set_ylabel(ylabel)
    axes.set_title(title)
    axes.hist(data, range=(0,60), bins=np.arange(62)-0.5, log=log, histtype='step', linewidth = '1.5');
    plt.grid(True, which='minor', axis='x', linewidth='0.5')
    return fig

In [ ]:
rechit = rootfile.arrays([b'rechit_thickness', b'rechit_energy',  b'rechit_layer',  b'rechit_time', \
                          b'rechit_x', b'rechit_y', b'rechit_z', b'rechit_eta', b'rechit_phi'])
rechit[b'rechit_x'].content[rechit[b'rechit_z'].content < 0] *= -1
NEvents = rechit[b'rechit_z'].shape[0]
simcluster = rootfile.arrays([b'simcluster_hits_indices',  b'simcluster_energy', b'simcluster_eta', b'simcluster_phi', b'simcluster_layers', b'simcluster_pid'])
#simcluster = rootfile.arrays([b'simcluster_hits_indices',  b'simcluster_energy', b'simcluster_eta', b'simcluster_phi', b'simcluster_layers'])

In [ ]:
sim_indices = awkward.fromiter(simcluster[b'simcluster_hits_indices'])
valid_sim_indices = sim_indices[sim_indices > -1]

In [ ]:
simcluster_rechit_cut = 3 #min no. of rechits in simcluster requirement (exclusive)
simcluster_mask = awkward.JaggedArray.fromcounts(valid_sim_indices.counts,valid_sim_indices.flatten().counts > simcluster_rechit_cut)
simcluster_mask = simcluster_mask & (simcluster[b'simcluster_energy'] > 1.0)
valid_sim_indices = valid_sim_indices[simcluster_mask]
for key, value in simcluster.items():
    if (key == b'simcluster_hits_indices'): continue
    simcluster[key] = awkward.fromiter(value)[simcluster_mask]

In [ ]:
valid_sim_indices_eventlevel = valid_sim_indices.flatten(1)
valid_sim_indices_eventlevel = awkward.fromiter(map(np.unique, valid_sim_indices_eventlevel))

In [ ]:
simmatched_rechit = {}
for key, value in rechit.items():
    simmatched_rechit[key] = value[valid_sim_indices_eventlevel]

In [ ]:
def rewrap_into_simcluster_structure(filelevel_array):
    return awkward.JaggedArray.fromcounts(valid_sim_indices.counts,\
        (awkward.JaggedArray.fromcounts(valid_sim_indices.content.counts, filelevel_array)))

rechit_simcluster = {}
select_rechit_simcluster = [b'rechit_energy', b'rechit_layer', b'rechit_eta', b'rechit_phi']
for key, value in rechit.items():
    if key not in select_rechit_simcluster: continue
    rechit_simcluster[key] = value[valid_sim_indices.flatten(1)]
    rechit_simcluster[key] = rewrap_into_simcluster_structure(rechit_simcluster[key].content)
    
#corrected_rechit_simcluster_energy = rechit_simcluster[b'rechit_energy'] * rewrap_into_simcluster_structure(np.take(absorber_weights,rechit_simcluster[b'rechit_layer'].content.content))
corrected_rechit_simcluster_energy = rechit_simcluster[b'rechit_energy']

## No. of rechits in Preprocessed Clusters

In [ ]:

preprocessed_clusters_pos = []
preprocessed_clusters_neg = []

preprocessed_clusters_pos_Nedges = []
preprocessed_clusters_neg_Nedges = []

for ievt in tqdm(range(NEvents)):
    g_pos = preprocessing_algo(rechit, valid_sim_indices, ievt = ievt, mask = rechit[b'rechit_z'][ievt] > 0,
                                   layered_norm = layer_norm, algo=grouping_algo, preprocessing_args=preprocessing_args)
    nx_graph_pos = nx.Graph()
    list_tmp = list(zip(find(g_pos.Ri)[0], find(g_pos.Ro)[0]))
    nx_graph_pos.add_edges_from(list_tmp)
    preprocessed_clusters_pos.append(awkward.fromiter(nx.connected_components(nx_graph_pos)))
    preprocessed_clusters_pos_Nedges.append(len(list_tmp))
    
    g_neg = preprocessing_algo(rechit, valid_sim_indices, ievt = ievt, mask = rechit[b'rechit_z'][ievt] < 0,
                                   layered_norm = layer_norm, algo=grouping_algo, preprocessing_args=preprocessing_args)
    nx_graph_neg = nx.Graph()
    list_tmp = list(zip(find(g_neg.Ri)[0], find(g_neg.Ro)[0]))
    nx_graph_neg.add_edges_from(list_tmp)
    preprocessed_clusters_neg.append(awkward.fromiter(nx.connected_components(nx_graph_neg)))
    preprocessed_clusters_neg_Nedges.append(len(list_tmp))

In [ ]:
preprocessed_clusters_pos = awkward.fromiter(preprocessed_clusters_pos)
preprocessed_clusters_neg = awkward.fromiter(preprocessed_clusters_neg)
#preprocessed_clusters_pos and preprocessed_clusters_neg are ids (within each pos/neg event) of rechits in each cluster

def rewrap_into_eventCluster_structure(filelevel_array, target_structure):
    return awkward.JaggedArray.fromcounts(target_structure.counts,\
        (awkward.JaggedArray.fromcounts(target_structure.flatten(0).counts, filelevel_array)))

rechit_idx_map_pos = awkward.fromiter(map(np.where, (rechit[b'rechit_z'] > 0))).flatten()
rechit_idx_map_neg = awkward.fromiter(map(np.where, (rechit[b'rechit_z'] < 0))).flatten()
#map from pos/neg id to event-level id

preprocessed_clusters_pos_gid = rewrap_into_eventCluster_structure(rechit_idx_map_pos[preprocessed_clusters_pos.flatten(1)].flatten(),\
                                  target_structure=preprocessed_clusters_pos)
preprocessed_clusters_neg_gid = rewrap_into_eventCluster_structure(rechit_idx_map_neg[preprocessed_clusters_neg.flatten(1)].flatten(),\
                                  target_structure=preprocessed_clusters_neg)

In [ ]:
preprocessed_clusters_counts = np.concatenate([preprocessed_clusters_pos_gid.flatten().counts, \
                                               preprocessed_clusters_neg_gid.flatten().counts])

In [ ]:
fig,axes = plt.subplots(figsize=(12, 7));
plotHist(axes, preprocessed_clusters_counts, "Rechits", "Preprocessed Clusters",\
         "No. of Rechits in Preprocessed Clusters", Nbins = 100, xlog=True, ylog=True)
figs.append(fig)

## No. of Edges in Events

In [ ]:
fig,axes = plt.subplots(figsize=(12, 7));
preprocessed_clusters_Nedges = np.concatenate((preprocessed_clusters_pos_Nedges, preprocessed_clusters_neg_Nedges))
plotHist(axes, preprocessed_clusters_Nedges, "N Edges", "Events",\
         "No. of Edges in Events", Nbins = 100, xlog=False, ylog=True)
figs.append(fig)

## Sum of Corrected Energy of Rechits in each Preprocessed Cluster

In [ ]:
#rechit_energy_corrected = rechit[b'rechit_energy'] * \
#    awkward.JaggedArray.fromcounts(rechit[b'rechit_energy'].counts,\
#                                   np.take(absorber_weights,rechit[b'rechit_layer'].flatten()))

rechit_energy_corrected = rechit[b'rechit_energy']

In [ ]:
rechit_energy_preprocessed_cluster_pos = rewrap_into_eventCluster_structure(rechit_energy_corrected[preprocessed_clusters_pos_gid.flatten(1)].flatten(),\
                                   preprocessed_clusters_pos_gid).flatten(0)
rechit_energy_preprocessed_cluster_neg = rewrap_into_eventCluster_structure(rechit_energy_corrected[preprocessed_clusters_neg_gid.flatten(1)].flatten(),\
                                   preprocessed_clusters_neg_gid).flatten(0)

rechit_energy_sum_preprocessed_cluster = np.concatenate([rechit_energy_preprocessed_cluster_pos.sum(), rechit_energy_preprocessed_cluster_neg.sum()])

In [ ]:
fig = plt.figure(figsize=(18,5));
ax1 = fig.add_subplot(121);
plotHist(ax1, rechit_energy_sum_preprocessed_cluster, "Sum of Energy of Rechits / GeV", "Preprocessed Clusters",\
         "Sum of Energy of Rechits in each Preprocessed Cluster", range=(rechit_energy_sum_preprocessed_cluster.min(), rechit_energy_sum_preprocessed_cluster.max()), Nbins = 100, xlog=False, ylog=True)
ax2 = fig.add_subplot(122);
plotHist(ax2, rechit_energy_sum_preprocessed_cluster, "Sum of Energy of Rechits / GeV", "Preprocessed Clusters",\
         "Sum of Energy of Rechits in each Preprocessed Cluster", Nbins = 100, xlog=True, ylog=True)
figs.append(fig)

## Preprocessed Cluster - First Layer Number


In [ ]:
rechit_layer_preprocessed_cluster_pos = rewrap_into_eventCluster_structure((rechit[b'rechit_layer'][preprocessed_clusters_pos_gid.flatten(1)]).flatten(),\
                                   target_structure=preprocessed_clusters_pos).flatten(0)
rechit_layer_preprocessed_cluster_neg = rewrap_into_eventCluster_structure((rechit[b'rechit_layer'][preprocessed_clusters_neg_gid.flatten(1)]).flatten(),\
                                   target_structure=preprocessed_clusters_neg).flatten(0)
rechit_layer_preprocessed_cluster = awkward.JaggedArray.concatenate([rechit_layer_preprocessed_cluster_pos,\
                                                                    rechit_layer_preprocessed_cluster_neg])

In [ ]:
figs.append(plotHist_layers(rechit_layer_preprocessed_cluster.min(),\
                            "Preprocessed Cluster", "Preprocessed Cluster - First Layer Number", xlabel= "First Layer Number"))

## Preprocessed Cluster - Last Layer Number

In [ ]:
figs.append(plotHist_layers(rechit_layer_preprocessed_cluster.max(),\
                            "Preprocessed Cluster", "Preprocessed Cluster - Last Layer Number", xlabel= "Last Layer Number"))

## Rechit Multiplicity in Greatest Intersection
A simcluster's "Greatest Intersection" is defined as the largest intersection of rechits between this simcluster and any preprocessed cluster (on pos/neg endcap respectively)

In [ ]:
def intersect_row(row):
    row = row.tolist()
    return np.intersect1d(row[0], row[1])

def intersect_table(awkward_table):
    return map(intersect_row, awkward_table)

intersection_pos = awkward.fromiter(map(intersect_table, valid_sim_indices[simcluster[b'simcluster_eta']>0].cross(preprocessed_clusters_pos_gid)))
intersection_neg = awkward.fromiter(map(intersect_table, valid_sim_indices[simcluster[b'simcluster_eta']<0].cross(preprocessed_clusters_neg_gid)))

intersection_pos = awkward.JaggedArray.fromcounts(valid_sim_indices[simcluster[b'simcluster_eta']>0].counts,\
    awkward.JaggedArray.fromcounts(np.repeat(preprocessed_clusters_pos_gid.counts, valid_sim_indices[simcluster[b'simcluster_eta']>0].counts), \
                       intersection_pos.flatten()))

intersection_neg = awkward.JaggedArray.fromcounts(valid_sim_indices[simcluster[b'simcluster_eta']<0].counts,\
    awkward.JaggedArray.fromcounts(np.repeat(preprocessed_clusters_neg_gid.counts, valid_sim_indices[simcluster[b'simcluster_eta']<0].counts), \
                       intersection_neg.flatten()))

#structure:
#intersection_pos/neg[event idx][simcluster idx (for pos/neg respectively)][idx of preprocessed cluster(for pos/neg respectively)]

In [ ]:
simcluster_energy_pos_neg = np.concatenate([simcluster[b'simcluster_energy'][simcluster[b'simcluster_eta']>0].flatten(),\
                                            simcluster[b'simcluster_energy'][simcluster[b'simcluster_eta']<0].flatten()])
simclusterEnergyCut = (simcluster_energy_pos_neg > 1.0)

In [ ]:
pos_simcluster_grtitsn_counts = np.array(list(map(lambda x: x.counts.max(), intersection_pos.flatten())))
neg_simcluster_grtitsn_counts = np.array(list(map(lambda x: x.counts.max(), intersection_neg.flatten())))
grtitsn_counts = np.concatenate([pos_simcluster_grtitsn_counts, neg_simcluster_grtitsn_counts])
fig,axes = plt.subplots(figsize=(12, 7));
plotHist(axes, grtitsn_counts[simclusterEnergyCut], "Rechits", "Simclusters",\
         "Rechits in Greatest Intersection for each Simcluster", Nbins = 100)
figs.append(fig)

In [ ]:
simcluster_counts = np.concatenate([valid_sim_indices[simcluster[b'simcluster_eta']>0].flatten().counts,\
                valid_sim_indices[simcluster[b'simcluster_eta']<0].flatten().counts])
nonzerocut = (simcluster_counts > 0)
fig,axes = plt.subplots(figsize=(12, 7));
plotHist(axes, grtitsn_counts[simclusterEnergyCut & nonzerocut]/simcluster_counts[simclusterEnergyCut & nonzerocut], "Ratio", "Simclusters",\
         "(Rechits in Greatest Intersection for each Simcluster) / (Rechits in Simcluter)", Nbins = 100)
figs.append(fig)

In [ ]:
map_tmp = awkward.JaggedArray.fromcounts(valid_sim_indices.counts,\
                                         list(zip(np.repeat(np.arange(valid_sim_indices.shape[0]),valid_sim_indices.counts), np.concatenate(list(map(np.arange, valid_sim_indices.counts))))))

map_posneg2event = np.concatenate([map_tmp[simcluster[b'simcluster_eta']>0].flatten(), map_tmp[simcluster[b'simcluster_eta']<0].flatten()])

## Activate these cells to explore Simclusters with low efficiency:

## Sum of Corrected Energy in Greatest Intersection

In [ ]:
grtitsn_pos = intersection_pos.flatten()[awkward.fromiter(map(lambda x: [np.argmax(x.counts)],intersection_pos.flatten()))].flatten()
grtitsn_neg = intersection_neg.flatten()[awkward.fromiter(map(lambda x: [np.argmax(x.counts)],intersection_neg.flatten()))].flatten()

In [ ]:
energy_grtitsn_pos = rewrap_into_eventCluster_structure(rechit_energy_corrected[awkward.JaggedArray.fromcounts(intersection_pos.counts, grtitsn_pos).flatten(1)].flatten(),\
                                   target_structure = awkward.JaggedArray.fromcounts(intersection_pos.counts, grtitsn_pos))
energy_grtitsn_neg = rewrap_into_eventCluster_structure(rechit_energy_corrected[awkward.JaggedArray.fromcounts(intersection_neg.counts, grtitsn_neg).flatten(1)].flatten(),\
                                   target_structure = awkward.JaggedArray.fromcounts(intersection_neg.counts, grtitsn_neg))

In [ ]:
h = np.concatenate([energy_grtitsn_pos.flatten().sum(), energy_grtitsn_neg.flatten().sum()])
fig = plt.figure(figsize=(18,5));
ax1 = fig.add_subplot(121);
plotHist(ax1, h[simclusterEnergyCut], "Sum of Energy in Greatest Intersection / GeV", "Simclusters",\
         "Sum of Energy in Greatest Intersection for each Simcluter", Nbins = 100)
ax2 = fig.add_subplot(122);
plotHist(ax2, h[simclusterEnergyCut]+0.00001, "Sum of Energy in Greatest Intersection / GeV", "Simclusters",\
         "Sum of Energy in Greatest Intersection for each Simcluter \n peak on left indicates 0 GeV", Nbins = 100, xlog=True)
figs.append(fig)

In [ ]:
simcluster_energy_posneg = np.concatenate([corrected_rechit_simcluster_energy[simcluster[b'simcluster_eta']>0].flatten().sum(),\
                                           corrected_rechit_simcluster_energy[simcluster[b'simcluster_eta']<0].flatten().sum()])
nonzeromask = (simcluster_energy_posneg > 0)

fig = plt.figure(figsize=(19,5));
ax1 = fig.add_subplot(121);
plotHist(ax1, h[simclusterEnergyCut & nonzeromask]/simcluster_energy_posneg[simclusterEnergyCut & nonzeromask], "Ratio of Energy", "Simclusters",\
         "(Sum of energy in Greatest Intersection)/(Sum of energy in Simcluster)", Nbins = 100)
ax2 = fig.add_subplot(122);
plotHist(ax2, h[simclusterEnergyCut & nonzeromask]/simcluster_energy_posneg[simclusterEnergyCut & nonzeromask] + 0.00001, "Ratio of Energy", "Simclusters",\
         "(Sum of energy in Greatest Intersection)/(Sum of energy in Simcluster) \n peak on left indicates 0", Nbins = 100, xlog=True)
figs.append(fig)

## Greatest Intersection - First Layer Number  

In [ ]:
layer_grtitsn_pos = rewrap_into_eventCluster_structure((rechit[b'rechit_layer'][awkward.JaggedArray.fromcounts(intersection_pos.counts, grtitsn_pos).flatten(1)]).flatten(),\
                                   target_structure = awkward.JaggedArray.fromcounts(intersection_pos.counts, grtitsn_pos)).flatten()
layer_grtitsn_neg = rewrap_into_eventCluster_structure((rechit[b'rechit_layer'][awkward.JaggedArray.fromcounts(intersection_neg.counts, grtitsn_neg).flatten(1)]).flatten(),\
                                   target_structure = awkward.JaggedArray.fromcounts(intersection_neg.counts, grtitsn_neg)).flatten()


In [ ]:
figs.append(plotHist_layers(np.concatenate([layer_grtitsn_pos.min(), layer_grtitsn_neg.min()])[simclusterEnergyCut],\
                            "Simcluster", "Greatest Intersection - First Layer Number", xlabel= "First Layer Number"))

## Greatest Intersection - Last Layer Number  

In [ ]:
figs.append(plotHist_layers(np.concatenate([layer_grtitsn_pos.max(), layer_grtitsn_neg.max()])[simclusterEnergyCut],\
                            "Simcluster", "Greatest Intersection - Last Layer Number", xlabel= "Last Layer Number"))

## Efficiency of Preprocessing Against Eta of Simcluster

In [ ]:
def plotHistRatio(axes, dataNumerator, dataNumeratorLabel, dataDenominator, dataDenominatorLabel, xlabel, ylabelNumerator, ylabelDenominator, title, xticks, Nbins = 100, range=None, ylog=False):
    fig = plt.figure(figsize=(12,8), constrained_layout=True);
    gs = fig.add_gridspec(5, 1)
    ax1 = fig.add_subplot(gs[:4, 0])
    ax1.set_ylabel(ylabelNumerator)
    ax1.set_title(title)
    h_numerator, bins,_ = ax1.hist(dataNumerator, bins=Nbins, range=range, histtype='step', log=ylog, label = dataNumeratorLabel); 
    h_denominator,_,_ = ax1.hist(dataDenominator, bins=bins, range=range, histtype='step', log=ylog, label = dataDenominatorLabel);
    plt.legend(loc='upper center')
    
    
    ax2 = fig.add_subplot(gs[4, 0], sharex=ax1);
    ax2.set_xlabel(xlabel)
    ax2.set_xticks(xticks, minor=True)
    ax2.set_ylim((0.0,1.0))
    ax2.set_ylabel(ylabelDenominator)
    #ax2.plot((bins[:-1] + bins[1:]) / 2, h_numerator / h_denominator)
    ax2.bar((bins[:-1] + bins[1:]) / 2, h_numerator / h_denominator, align='center', width=bins[1] - bins[0], fill=False)
    plt.grid(True, which='major')
    #print((bins[:-1] + bins[1:]) / 2)
    return fig;

In [ ]:
nonzerocut = (simcluster_counts > 0)
efficiency_simcluster_idx = np.where(grtitsn_counts[simclusterEnergyCut & nonzerocut]/simcluster_counts[simclusterEnergyCut & nonzerocut] > 0.9)[0]

In [ ]:
simcluster_eta_pos_neg = np.concatenate([simcluster[b'simcluster_eta'][simcluster[b'simcluster_eta']>0].flatten(),\
simcluster[b'simcluster_eta'][simcluster[b'simcluster_eta']<0].flatten()])

fig = plotHistRatio(axes, simcluster_eta_pos_neg[simclusterEnergyCut & nonzerocut][efficiency_simcluster_idx], "Rechits_Preprocessed/Rechits_Simcluster > 0.9",\
              simcluster_eta_pos_neg[simclusterEnergyCut & nonzerocut], "All Simclusters" , "Eta", "Simclusters", "Efficiency",\
         "Efficiency against Eta", Nbins = 100, xticks=np.arange(-3,3,0.1))
figs.append(fig)


## Efficiency of Preprocessing Against Phi of Simcluster

In [ ]:
simcluster_phi_pos_neg = np.concatenate([simcluster[b'simcluster_phi'][simcluster[b'simcluster_eta']>0].flatten(),\
simcluster[b'simcluster_phi'][simcluster[b'simcluster_eta']<0].flatten()])

fig = plotHistRatio(axes, simcluster_phi_pos_neg[simclusterEnergyCut & nonzerocut][efficiency_simcluster_idx], "Rechits_Preprocessed/Rechits_Simcluster > 0.9",\
              simcluster_phi_pos_neg[simclusterEnergyCut & nonzerocut], "All Simclusters" , "Phi", "Simclusters", "Efficiency",\
         "Efficiency against Phi", Nbins = 100, xticks=np.arange(-3,3,0.1))
figs.append(fig)



## Efficiency of Preprocessing Against Energy of Simcluster

In [ ]:
simcluster_energy_pos_neg = np.concatenate([simcluster[b'simcluster_energy'][simcluster[b'simcluster_eta']>0].flatten(),\
simcluster[b'simcluster_energy'][simcluster[b'simcluster_eta']<0].flatten()])

fig = plotHistRatio(axes, simcluster_energy_pos_neg[simclusterEnergyCut & nonzerocut][efficiency_simcluster_idx], "Rechits_Preprocessed/Rechits_Simcluster > 0.9",\
              simcluster_energy_pos_neg[simclusterEnergyCut & nonzerocut], "All Simclusters" , "Simcluster Energy", "Simclusters", "Efficiency",\
         "Efficiency against Simcluster Energy", Nbins = 100, xticks=np.arange(0,100,1.0))
figs.append(fig)




## Output

In [ ]:
import matplotlib.backends.backend_pdf
outname = 'validation_preprocessing_' + fname.rstrip('.root').split('/')[-1] +'.pdf'
pdf = matplotlib.backends.backend_pdf.PdfPages(outname)
for fig in figs: 
    pdf.savefig(fig)
pdf.close()